In [2]:
pip -V

pip 25.0 from /users/k23070952/.local/lib/python3.9/site-packages/pip (python 3.9)
Note: you may need to restart the kernel to use updated packages.


In [1]:
cd ~/MultiFidelity-ProcessOpt/Perovskites/

/cephfs/volumes/hpc_home/k23070952/a774a32b-b8f8-42f8-b661-406a5dd49fd8/MultiFidelity-ProcessOpt/Perovskites


In [1]:
pip list

Package                       Version                 Editable project location
----------------------------- ----------------------- --------------------------------------------------------------------------------------------------------------------
about-time                    4.2.1
absl-py                       2.1.0
accelerate                    1.5.1
aiohappyeyeballs              2.4.3
aiohttp                       3.11.7
aiosignal                     1.3.1
alabaster                     0.7.12
alembic                       1.14.1
alive-progress                3.2.0
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
annotated-types               0.7.0
ansicolors                    1.1.8
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array_record                  0.5.1
arrow                         1.2.2
astroid               

In [2]:
from olympus import Surface, Campaign
from atlas.planners.gp.planner import GPPlanner

In [3]:
from olympus.datasets import Dataset

In [4]:
#!/usr/bin/env python

import json
import pickle
import numpy as np
import pandas as pd
from copy import deepcopy

from olympus.datasets import Dataset
from olympus.objects import (
	ParameterContinuous,
	ParameterDiscrete, 
	ParameterCategorical, 
	ParameterVector
)
from olympus.campaigns import ParameterSpace, Campaign

from atlas.planners.multi_fidelity.planner import MultiFidelityPlanner



In [19]:
# config
dataset = Dataset(kind='perovskites')
NUM_RUNS = 2
# BUDGET = 30
COST_BUDGET = 50 # 200.
NUM_INIT_DESIGN = 10
NUM_CHEAP = 8

# lookup table
# organic --> cation --> anion --> bandgap_hse06/bandgap_gga
LOOKUP = pickle.load(open('0.Data/lookup_table.pkl', 'rb'))
# print(lookup.keys())
# print(lookup['Ethylammonium']['Ge']['F'].keys())



In [20]:
dataset.param_space

In [21]:

def build_options_descriptors(json_file):
	with open(json_file, 'r') as f:
		content = json.load(f)

	options = list(content.keys())
	descriptors = [list(content[opt].values()) for opt in options]

	return options, descriptors

def measure(params, s):
	# high-fidelity is hse06, low-fidelity is gga
	if s == 1.0:
		measurement = np.amin(
			LOOKUP[params.organic.capitalize()][params.cation][params.anion]['bandgap_hse06']
		)
	elif s == 0.1:
		measurement = np.amin(
			LOOKUP[params.organic.capitalize()][params.cation][params.anion]['bandgap_gga']
		)
	return measurement

def get_min_hse06_bandgap(param_space):
	organic_options = [o.capitalize() for o in param_space[1].options]
	cation_options = [o.capitalize() for o in param_space[2].options]
	anion_options = [o.capitalize() for o in param_space[3].options]

	hse06_bandgaps = []
	for organic_option in organic_options:
		for cation_option in cation_options:
			for anion_option in anion_options:
				hse06_bandgaps.append(
					np.amin(
						LOOKUP[organic_option][cation_option][anion_option]['bandgap_hse06']
					)
				)
	min_hse06_bandgap = np.amin(hse06_bandgaps)
	return min_hse06_bandgap

def compute_cost(params):
	costs = params[:,0].astype(float)
	return np.sum(costs)



In [22]:

# build parameter space
param_space = ParameterSpace()

# fidelity param
param_space.add(ParameterDiscrete(name='s', options=[0.1, 1.0], low=0.1, high=1.0))
for param in dataset.param_space: # add perovskite component parameters ('organic', 'cation', and 'anion')
	print(param)
	param_space.add(param)


all_data = []
min_hse06_bandgap = get_min_hse06_bandgap(param_space)
print('MIN HSE06 BANDGAP : ', min_hse06_bandgap)
# >>> MIN HSE06 BANDGAP :  1.5249
# hydrazinium	Sn	I



Categorical (name='organic', num_opts: 16, options=['ethylammonium', 'propylammonium', 'butylammonium', 'isopropylammonium', 'dimethylammonium', 'acetamidinium', 'methylammonium', 'guanidinium', 'hydroxylammonium', 'formamidinium', 'tetramethylammonium', 'hydrazinium', 'ammonium', 'trimethylammonium', 'azetidinium', 'imidazolium'], descriptors=[[-0.460099995136261, -0.2239799946546554, 1.3964999914169312, -1.8414201736450195, 1.2615654468536377, 69.08699798583984], [-0.460099995136261, -0.2239799946546554, 1.3964999914169312, -1.8414201736450195, 1.2615654468536377, 69.08699798583984], [-0.460099995136261, -0.2239799946546554, 1.3964999914169312, -1.8414201736450195, 1.2615654468536377, 69.08699798583984], [-0.460099995136261, -0.2239799946546554, 1.3964999914169312, -1.8414201736450195, 1.2615654468536377, 69.08699798583984], [-0.460099995136261, -0.2239799946546554, 1.3964999914169312, -1.8414201736450195, 1.2615654468536377, 69.08699798583984], [-0.460099995136261, -0.22397999465465

In [23]:

for run_ix in range(NUM_RUNS):

	campaign = Campaign()
	campaign.set_param_space(param_space)

	planner = MultiFidelityPlanner(
		goal='minimize',
		init_design_strategy='random',
		num_init_design=NUM_INIT_DESIGN,
		use_descriptors=True,
		batch_size=1,
		acquisition_optimizer_kind='pymoo',
		fidelity_params=0,
		fidelities=[0.1, 1.],
	)

	planner.set_param_space(param_space)

	COST = 0.

	target_rec_measurements = []
	iter_ = 0
	while COST < COST_BUDGET:
		print(f'\nRUN : {run_ix+1}/{NUM_RUNS}\tITER : {iter_+1}\tCOST : {COST}\n')

		if iter_ % NUM_CHEAP == 0:
			planner.set_ask_fidelity(1.0)
		else:
			planner.set_ask_fidelity(0.1)

		samples = planner.recommend(campaign.observations)
		for sample in samples:
			measurement = measure(sample, sample.s)
			campaign.add_observation(sample, measurement)

			print('SAMPLE : ', sample)
			print('MEASUREMENT : ', measurement)

			iter_+=1

		# do a check to see if model will find the optimal
		if campaign.num_obs > NUM_INIT_DESIGN:
			# make greedy recommendation on the target fidelity
			rec_sample = planner.recommend_target_fidelity(batch_size=1)[0]
			rec_measurement = measure(rec_sample, rec_sample.s)
			print('')
			print('REC SAMPLE : ', rec_sample)
			print('REC MEASUREMENT : ', rec_measurement)
			print('')
			target_rec_measurements.append(rec_measurement)
			# kill the run if we have found the lowest hse06 bandgap
			# on the most recent high-fidelity measurement
			if rec_measurement == min_hse06_bandgap:
				print('found the min hse06 bandgap!')
				break
		else:
			target_rec_measurements.append(measurement)
			# kill the run if we have found the lowest hse06 bandgap
			# on the most recent high-fidelity measurement
			if measurement == min_hse06_bandgap and samples[0].s == 1.:
				print('found the min hse06 bandgap!')
				break

		
		COST = compute_cost(params=campaign.observations.get_params())



	s_col = campaign.observations.get_params()[:, 0]
	x0_col = campaign.observations.get_params()[:, 1]
	x1_col = campaign.observations.get_params()[:, 2]
	x2_col = campaign.observations.get_params()[:, 3]

	obj0_col = np.array(target_rec_measurements) #campaign.observations.get_values()

	data = pd.DataFrame({
		's': s_col,
		'organic': x0_col,
		'cation': x1_col,
		'anion': x2_col,
		'obj': obj0_col,
	})
	all_data.append(data)
	pickle.dump(all_data, open('mf_results.pkl', 'wb'))

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

...

/users/k23070952/.local/lib/python3.9/site-packages/botorch/models/gpytorch.py:477: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 3 and Y with dimension 4).
  self._validate_tensor_args(X=X, Y=Y, Yvar=noise, strict=False)
/users/k23070952/.local/lib/python3.9/site-packages/botorch/models/gpytorch.py:239: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 3 and Y with dimension 4).
  self._validate_tensor_args(X=X, Y=Y, Yvar=Yvar, strict=False)
/users/k23070952/.local/lib/python3.9/site-packages/pymoo/operators/repair/rounding.py:18: RuntimeWarning: invalid value encountered in cast
  return np.around(X).astype(int)
/users/k230709

SAMPLE :  ParamVector(s = 0.1, organic = butylammonium, cation = Sn, anion = F)
MEASUREMENT :  3.7624


────────────────────── Acquisition function optimization ───────────────────────

ATLAS [INFO] Optimizing acquisition function with pymoo GA for 1 fixed parameters...

KeyboardInterrupt: 